* gene1: Mirlet7g 

annotation data 에서 Mirlet7g 위치 찾기

In [5]:
!grep -i mirlet7g ../datapack/gencode.gtf

chr9	ENSEMBL	gene	106056039	106056126	.	+	.	gene_id "ENSMUSG00000065440.3"; gene_type "miRNA"; gene_name "Mirlet7g"; level 3; mgi_id "MGI:2676800";
chr9	ENSEMBL	transcript	106056039	106056126	.	+	.	gene_id "ENSMUSG00000065440.3"; transcript_id "ENSMUST00000083506.3"; gene_type "miRNA"; gene_name "Mirlet7g"; transcript_type "miRNA"; transcript_name "Mirlet7g-201"; level 3; transcript_support_level "NA"; mgi_id "MGI:2676800"; tag "basic";
chr9	ENSEMBL	exon	106056039	106056126	.	+	.	gene_id "ENSMUSG00000065440.3"; transcript_id "ENSMUST00000083506.3"; gene_type "miRNA"; gene_name "Mirlet7g"; transcript_type "miRNA"; transcript_name "Mirlet7g-201"; exon_number 1; exon_id "ENSMUSE00000522665.2"; level 3; transcript_support_level "NA"; mgi_id "MGI:2676800"; tag "basic";


CLIP-seq결과에서 Mirlet7g 위치로 추출하기

In [26]:
!samtools view -b -o CLIP-let7g.bam ../datapack/CLIP-35L33G.bam chr9:106056039-106056126
!samtools view CLIP-let7g.bam | wc -l

     163


_근데, 이렇게 chr위치를 지정해서 뽑은 결과가 왜 그 위치 바깥에 잇는 read까지 불러냈는지 잘 모르겠다._

In [9]:
!samtools view CLIP-let7g.bam | head -3

SRR458758.222778	16	chr9	106007092	38	8M2D2M68520N16M1S	*	0	0	AGCAATAGCCCAAATCAGGGCTCATTT	<AE?E=??8BB?DGBGG=4<?:477>=	MD:Z:8^AA2T15	NH:i:1	HI:i:1	NM:i:3	SM:i:38	XQ:i:40	X2:i:0	XO:Z:UU	XS:A:-
SRR458758.2480600	16	chr9	106007092	40	8M2D2M68520N16M1S	*	0	0	AGCAATAGCCCAAATCAGGGCTCATTT	IFHHHHHIIHHIIIHIIIHIEGIIGI?	MD:Z:8^AA2T15	NH:i:1	HI:i:1	NM:i:3	SM:i:40	XQ:i:40	X2:i:0	XO:Z:UU	XS:A:-
SRR458758.3458120	16	chr9	106007092	39	8M2D2M68520N16M1S	*	0	0	AGCAATAGCCCAAATCAGGGCTCATTT	FGHDHGHEBCEEEBHGHFGFEGDGGGB	MD:Z:8^AA2T15	NH:i:1	HI:i:1	NM:i:3	SM:i:39	XQ:i:40	X2:i:0	XO:Z:UU	XS:A:-


_내가 파악해본 바에 의하면 파일업은 그 싱글 레퍼런스포지션별로 그 베이스의 파일업, 리드커버리지를 정리해서 보여주는 파일인거 같음_  
_The SAMtools mpileup utility provides a summary of the coverage of mapped reads on a reference sequence at a single base pair resolution._

In [10]:
!samtools mpileup CLIP-let7g.bam > CLIP-let7g.pileup
!wc -l CLIP-let7g.pileup

[mpileup] 1 samples in 1 input files
   68548 CLIP-let7g.pileup


pileup format은 다음과같다고 한다.
* 1st column: Chromosome name
* 2nd column: 1-based position on the chromosoem
* 3rd column: Ref. base at this position (우리는 pileup할 때 -f 옵션을 안주었으므로 이건 다 N나옴)
* 4th column: number of reads covering this position
* 5th column: basereads
* 6th column: quality

_그럼 이 파일업 파일의 헤드랑 테일을 봤을 때 사실은 106,056,039 부터 106,056,126 위치의 결과가 나와야하는데 보니까 엄청 길게 나옴. 즉 68,548 bp의 파일업결과가 잇는거임 (106,075,639-106,007,092 +1)_  
_이 문제가 위에서 나왔던 samtool에서 chr:위치 로 추출할 때 그 바깥까지 같이 찾은거랑 같은 맥락의 문제인 듯_

In [23]:
!head -3 CLIP-let7g.pileup ;tail -3 CLIP-let7g.pileup

chr9	106007092	N	9	^Ga^Ia^Ha^Ia^Ga^Ga^Ha^Ia^Ia	<IFIGGHII
chr9	106007093	N	9	ggggggggg	AFGIEGDII
chr9	106007094	N	9	ccccccccc	EHHIGAFEH
chr9	106075637	N	9	aaaaaaaaa	7IGIGBCGI
chr9	106075638	N	9	ttttttttt	7GGIGDEGI
chr9	106075639	N	9	t$t$t$t$t$t$t$t$t$	>IGFDDDGH


위에서 봤던거처럼 Mirlet7g의 위치인 106056039와 106056126 에 맵핑되는 결과들만 가져오기

In [19]:
!awk '$2 >= 106056039 && $2 <= 106056126 {print $0; }' CLIP-let7g.pileup > CLIP-let7g-gene.pileup
!wc -l CLIP-let7g-gene.pileup

      88 CLIP-let7g-gene.pileup


In [20]:
!tail CLIP-let7g-gene.pileup

chr9	106056117	N	138	<<<<<<<<<CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC	BHEIG?DIIHGHII:;HGIIDGGGIIBGEB?>H<GGIIIIDIIGFHIHH9IGG=GGHIDH?DGIBIIHHGGIEIGI8GIDHDG.GGHEDIIDIDGDHIDGIFGHG;DCDDHEE@I?CGG:IHGIBGIHIIDG@DHIGG
chr9	106056118	N	139	<<<<<<<<<CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC	BHEIG?DIIHGHII3AHIIIDGGGIIB3EHFBH>GIIIIIIIIIHIIHHAIIGBDGHIDH?GIIGIIHHDIIHIGIGGI=BGG9GIHHHIIGI=IAIIGGIEHGIEBG@DHDE;IBDGG4IGGIGGIIIIGGGDGIH1G
chr9	106056119	N	138	<<<<<<<<<TTTTTTTTTTTTTTTTTTTTTTTTTT$TTTT$TTTTTTTTTTTTTTT$TTTT$TTTTTTT$TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT	BHEIG?DIIGEFII:/GIIEDDHDIIF>BH3<HBEGIHIGFIIIBIIBE:HHGD>EGGDH7BIIGGIHHDHIHDEIGEIDHGG;GIHGHHIGD:IGHD:GGEHH?3G87HD</I=DGG=IHHIHBIFII@<HDHHH1G
chr9	106056120	N	133	<<<<<<<<<TTTTTTTTTTTTTTTTTTTTTTTTTTTT$TTTT$TTT$TTTTTTTTTT$TTTTTTTT$TTTTTTTT

In [27]:
import pandas as pd
pileup_7g = pd.read_csv("CLIP-let7g-gene.pileup",
                        sep ='\t', names = ["chrom", "pos", "_ref", "count", "basereads", "quals"])
pileup_7g.tail()

,chrom,pos,_ref,count,basereads,quals
83,chr9,106056122,N,88,<<<<<<<<<C$C$C$C$C$C$C$C$C$C$C$C$C$CCCCCC$C$C$...,BHEIG?DIIEEGIIC;GIHEGBIIIIB1=FII?FEIGGGHDBIG=H...
84,chr9,106056123,N,31,<<<<<<<<<CCCCCCCCCCCCCCCCCCCCCC,BHEIG?DIIIIII>GIGGIGGD>BIHHHIEH
85,chr9,106056124,N,31,<<<<<<<<<AAAAAAAAAAAAAAAAAAAAAA,BHEIG?DIIIIHIGGIGGGIG:9DDBIEGFH
86,chr9,106056125,N,31,<<<<<<<<<GGGGGGGGGGGGGGGGGGGGGG,BHEIG?DIIIIIIGGE@GFIGD;GIGIIFHD
87,chr9,106056126,N,30,<<<<<<<<<GGGGGGGGGGGGGGGGGGGGG,BHEIG?DIIIIGHGHIGHI>G;GGGIGIHG


In [28]:
import re
toremove = re.compile('[<>*#^$]')
pileup_7g["matches"] = pileup_7g["basereads"].apply(lambda x: toremove.sub("", x))

In [29]:
pileup_7g[["chrom", "pos", "matches"]]

,chrom,pos,matches
0,chr9,106056039,
1,chr9,106056040,
2,chr9,106056041,
3,chr9,106056042,
4,chr9,106056043,
...,...,...,...
83,chr9,106056122,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...
84,chr9,106056123,CCCCCCCCCCCCCCCCCCCCCC
85,chr9,106056124,AAAAAAAAAAAAAAAAAAAAAA
86,chr9,106056125,GGGGGGGGGGGGGGGGGGGGGG


_그러면 이제 예시로 Mirlet7g gene에서 106056094위치의 matched or substituted coverage를 확인해보자_

In [37]:
pileup_7g.to_excel("test.xlsx")

In [34]:
pileup_7g[pileup_7g["pos"] == 106056097].iloc[0]["matches"]

'GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG'

1. 각 position별로 base수를 셉니다.
2. 각 position별로 Shannon entropy를 계산합니다.
3. 계산한 Shannon entropy를 bedgraph format으로 출력합니다. 말은 복잡해도 실제로는 간단합니다. 4칸짜리를 만드시면 됩니다.
4. 결과 파일을 본인의 Google Drive에서 가져옵니다.
5. UCSC Genome Browser에 접속해서 Genome은 mm39로 선택합니다.
6. 그래프 아랫쪽의 add custom tracks 버튼을 누릅니다.
7. Paste URLs or data 옆의 Choose File 버튼을 누르고 8. 우리가 만든 bedgraph 파일을 업로드 합니다.
9. 그리고 이리 저리 감상해 보다가 View 메뉴의 PDF/PS 를 눌러서 인증샷을 한 번 찍습니다.
10. Mirlet7d와 Mirlet7f-1도 한 번 해 봅니다.